# Eric González Caballero - MSc Big Data Analytics Thesis
## Forecasting the System Imbalance in the Spanish Electricity Market


### Notebook 02 - DataTransformation

This notebook aims to process and enrich the raw ingested data by deriving features that can be used for a ML model. It will save the output in the curated path.

#### Libraries import

In [11]:
# Import general libraries
import numpy as np
import pandas as pd

import datetime
import holidays

import sys
sys.path.append('../libraries')

# Import custom libraries
from utils import *

#### Constants definition

In [12]:
# Dynamic dates, time in UTC+01/02 (Spanish local time, not utc)
now = datetime.datetime.now()
today = datetime.date.today()

yesterday = today - datetime.timedelta(days=1)
tomorrow = today + datetime.timedelta(days=1)

# Set desired dates to be used in the correct format
start = "2019-01-01T00:00:00"
end = tomorrow.strftime("%Y-%m-%dT23:00:00")

# I/O Paths
input_path = "../data/curated/data_bronze.csv"
output_path = "../data/curated/data_silver.csv"

#### Input

In [13]:
df = pd.read_csv(input_path, index_col=0)
df

,Previsión diaria D+1 demanda,Demanda real,Demanda programada,Previsión diaria D+1 fotovoltaica,Previsión diaria D+1 eólica,Precio mercado SPOT Diario
datetime,,,,,,
2019-01-01 00:00:00+01:00,23753.0,23676.0,23251.0,0.0,3214.0,66.88
2019-01-01 01:00:00+01:00,23018.0,23128.0,22485.0,0.0,3222.0,66.88
2019-01-01 02:00:00+01:00,21808.0,22109.0,20977.0,0.0,3081.0,66.00
2019-01-01 03:00:00+01:00,20635.0,20666.0,19754.0,0.0,3069.0,63.64
2019-01-01 04:00:00+01:00,19824.0,19680.0,19321.0,0.0,2973.0,58.85
...,...,...,...,...,...,...
2022-08-10 19:00:00+02:00,31688.3,NaN,NaN,4284.4,4608.0,153.15
2022-08-10 20:00:00+02:00,31126.5,NaN,NaN,1513.6,4780.5,166.50
2022-08-10 21:00:00+02:00,31363.5,NaN,NaN,99.6,5048.3,171.68


#### Transformations

In [14]:
# Change to local time (no utc) so that index is parsed into DateIndex type (enables nice pandas features for time series)
index = pd.Series(df.index)
index = index.apply(lambda x: pd.to_datetime(x, utc=False).tz_localize(None))
df.index = index

In [15]:
# Add the imbalance column as a function of scheduled and real demands
df.loc[np.isnan(df['Demanda programada'] - df['Demanda real']), 'Signo del desvío'] = np.nan
df.loc[df['Demanda programada'] >= df['Demanda real'], 'Signo del desvío'] = 1
df.loc[df['Demanda programada'] < df['Demanda real'], 'Signo del desvío'] = -1

# Derive 24/48h lagged variables  
# df['Signo_lag_1h'] = df['Signo del desvío'].shift(1)
df['Signo_lag_24h'] = df['Signo del desvío'].shift(24)
# df['Signo_lag_36h'] = df['Signo del desvío'].shift(36)
df['Signo_lag_48h'] = df['Signo del desvío'].shift(48)

# Remove original columns
df = df.drop('Demanda programada', 1)
df = df.drop('Demanda real', 1)

Flag if a certain day was holiday or not, source of the function: https://github.com/nicholasjhana/short-term-energy-demand-forecasting/blob/master/data_creation_day_types.ipynb

In [16]:
# Get holidays flagged df
holidays = get_holidays(start=start, stop=end).astype(int)
holidays.index.name = 'datetime'

# Join the data df with holidays df
df = df.join(holidays)

In [17]:
# Derive more granular datetime columns
df['Year'] = df.index
df['Month'] = df.index
df['Day'] = df.index
df['Weekday'] = df.index
df['Hour'] = df.index

df['Year'] = df['Year'].apply(lambda x: x.year)
df['Month'] = df['Month'].apply(lambda x: x.month)
df['Day'] = df['Day'].apply(lambda x: x.day)
df['Weekday'] = df['Weekday'].apply(lambda x: x.weekday())
df['Hour'] = df['Hour'].apply(lambda x: x.hour)

# In case an index reset to integers is needed
# df['datetime'] = df.index
# df.reset_index(drop=True, inplace=True)

df.index.name = "DateIndex"

df

,Previsión diaria D+1 demanda,Previsión diaria D+1 fotovoltaica,Previsión diaria D+1 eólica,Precio mercado SPOT Diario,Signo del desvío,Signo_lag_24h,Signo_lag_48h,Holiday,Year,Month,Day,Weekday,Hour
DateIndex,,,,,,,,,,,,,
2019-01-01 00:00:00,23753.0,0.0,3214.0,66.88,-1.0,NaN,NaN,1,2019,1,1,1,0
2019-01-01 01:00:00,23018.0,0.0,3222.0,66.88,-1.0,NaN,NaN,1,2019,1,1,1,1
2019-01-01 02:00:00,21808.0,0.0,3081.0,66.00,-1.0,NaN,NaN,1,2019,1,1,1,2
2019-01-01 03:00:00,20635.0,0.0,3069.0,63.64,-1.0,NaN,NaN,1,2019,1,1,1,3
2019-01-01 04:00:00,19824.0,0.0,2973.0,58.85,-1.0,NaN,NaN,1,2019,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10 19:00:00,31688.3,4284.4,4608.0,153.15,NaN,-1.0,1.0,0,2022,8,10,2,19
2022-08-10 20:00:00,31126.5,1513.6,4780.5,166.50,NaN,NaN,1.0,0,2022,8,10,2,20
2022-08-10 21:00:00,31363.5,99.6,5048.3,171.68,NaN,NaN,1.0,0,2022,8,10,2,21


#### Output

In [18]:
# Save into curated as the latest file
df.to_csv(output_path, index=True)